In [ ]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import glob
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask, atsa_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt
import gc


In [ ]:
os.environ.update({'OMP_NUM_THREADS': '1',
                  'MKL_NUM_THREADS': '1',
                  'OPENBLAS_NUM_THREADS': '1'})

In [ ]:

wkDIR = os.getcwd()
outDIR_base = 'data/hls_nrt/'

cluster_loc = 'hpc'

In [ ]:
yr = 2013
os.chdir(wkDIR)
#prefix = 'redtop' 
prefix = 'cper'

full_smooth = True

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = os.path.join(outDIR_base, prefix)

mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
mod_cov = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

In [ ]:

if cluster_loc == 'local':
    os.chdir(wkDIR)
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 30, 'x': 30, 'time':-1}
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    wkDIR = '/project/cper_neon_aop/hls_nrt/'
    outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix
    os.chdir(wkDIR)
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up moderate cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=20
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

In [ ]:
t00 = time.time()
nc_flist = list(filter(os.path.isfile, 
                    glob.glob(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/' + 'hls_ds_' + str(yr) + '*.nc'))))
if len(nc_flist) > 0:
    nc_flist.sort(key=lambda x: os.path.getctime(x))
    nc_f = nc_flist[-1]
else:
    print('ERROR: No temp nc file found')

hls_ds = xr.open_dataset(nc_f).load()
#hls_atsa = xr.open_dataset(re.sub('hls_ds_', 'hls_atsa_', nc_f))

In [ ]:
hls_ds

In [ ]:
if len(hls_ds['time'].groupby('tile_id').apply(lambda x: x.drop_duplicates('time', False))) < len(hls_ds['time']):
    print('    reducing along id dimension to single observation for each date, keeping least-masked image')
    print('NOTE: THIS SHOULD NOT BE NEEDED! GO BACK AND CHECK HLS2_aoi_yr_download.ipynb')
    hls_ds = hls_ds.groupby('tile_id').apply(lambda x: x.sortby('maskcov_pct').drop_duplicates('time', keep='first')).sortby('time').compute()

In [ ]:
#hls_bolton_mask = bolton_mask(hls_ds).compute()
#hls_mask_snow = mask_hls(hls_ds['FMASK'], mask_types=['snow'])
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])

#show((hls_bolton.sum(dim='time') / len(hls_mask.time)) > 0.1)

In [ ]:
if len(np.unique(hls_ds.tile_id)) > 1:
    hls_atsa = hls_ds.groupby('tile_id').apply(lambda x: atsa_mask(x.where(
        x['BLUE'].notnull(), drop=True))).compute()
    hls_atsa = hls_atsa.transpose('time', 'y', 'x')
else:
    hls_ds = hls_ds.reset_coords(drop=True)
    try:
        hls_atsa = atsa_mask(hls_ds).compute()
        mask_atsa = True
    except ValueError:
        print('WARNING: Could not compute ATSA cloud/shadow mask')
        mask_atsa = False

In [ ]:
client.restart()

In [ ]:
hls_ds = xr.merge([hls_ds, hls_atsa], join='inner')
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])
hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['cirrus',
                                                'cloud',
                                                'cloud_adj',
                                                'shadow', 
                                                'snow',
                                                'water'])
#hls_mask = mask_hls(hls_ds['FMASK'], 'all')
hls_ds = hls_ds.where(hls_mask == 0)

In [ ]:
if mask_atsa:
    print('masking out clouds and shadows detected by ATSA')
    hls_ds = hls_ds.where(hls_ds['ATSA'] == 1)

In [ ]:
if 'tile_id' in hls_ds.coords and len(hls_ds.tile_id) > 1:
    hls_ds = hls_ds.groupby('time').mean()

In [ ]:
t0 = time.time()
#print('   applying tertiary cloud mask...')
#hls_ds = hls_ds.chunk({'time': -1, 'y': 100, 'x': 100})
t1 = time.time()
#hls_bolton_mask = bolton_mask(hls_ds).compute()
#hls_ds = hls_ds.where(hls_bolton_mask == 0, drop=True)
# mask out scenes where < 75% of the region is cloud-free
partial_mask = (hls_ds['NIR1'].isnull().sum(dim=['y', 'x'])/np.product(hls_ds['NIR1'].shape[1:])) < 0.25
partial_mask = partial_mask#.persist()
hls_ds = hls_ds.where(partial_mask, drop=True).compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

In [ ]:
hls_ds

In [ ]:
client.restart()

In [ ]:
if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

In [ ]:
hls_ds = hls_ds.chunk({'time': 1, 'y': -1, 'x': -1})
print('   computing NDVI for available dates...')
# create ndvi array
xr_ndvi = ndvi_func(hls_ds)#.compute()
xr_ndvi.name = 'NDVI'


print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_ndvi['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_ndvi['time'].values],
                             'x': xr_ndvi.x,
                             'y': xr_ndvi.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['NDVI'], xr_ndvi], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr

dat_out_ds = dat_out_ds.chunk({'time': -1, 'y': 20, 'x': 20})

if full_smooth:
    dat_out_da = despike_ts_xr(dat_out_ds['NDVI'],
                               dat_thresh=0.07, 
                               mask_outliers=False,
                               iters=2,
                               dims=['time'])
    dat_out_da = smooth_xr(dat_out_da, 
                           dims=['time'], 
                           kwargs={'double': True, 'limit': 91})
else:
    dat_out_da = dat_out_ds['NDVI'].interpolate_na('time').rolling(time=21, 
                                                                   center=True,
                                                                   min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
    
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

In [ ]:
print('   computing biomass for available dates...')
# create biomass array
xr_bm = hls_ds.map_blocks(pred_bm, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm.name = 'Biomass'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm['time'].values],
                             'x': xr_bm.x,
                             'y': xr_bm.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass'], xr_bm], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], xr_bm], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})

if full_smooth:
    dat_out_da = despike_ts_xr(dat_out_ds['Biomass'],
                               dat_thresh=150, 
                               mask_outliers=True,
                               z_thresh=5.0,
                               iters=2,
                               dims=['time'])
    dat_out_da = smooth_xr(dat_out_da, 
                           dims=['time'], 
                           kwargs={'double': True, 'limit': 91})
else:
    dat_out_da = dat_out_ds['Biomass'].interpolate_na('time').rolling(time=21, 
                                                                      center=True,
                                                                      min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()

dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

In [ ]:
print('   computing biomass SE for available dates...')
# create biomass SE array
xr_bm_se = hls_ds.map_blocks(pred_bm_se, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm_se.name = 'Biomass_SE'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm_se['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm_se['time'].values],
                             'x': xr_bm_se.x,
                             'y': xr_bm_se.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass_SE=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})

dat_out_da = dat_out_ds['Biomass_SE'].interpolate_na('time').rolling(time=21, 
                                                                     center=True,
                                                                     min_periods=1).mean()

dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()

dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

In [ ]:
print('   computing cover for available dates...')
# create cover array
xr_cov = pred_cov(hls_ds, 
                  model=mod_cov).where(hls_ds['NIR1'].notnull()).compute()

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_cov['time'].max().values]

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    # create an output Dataset template with all dates
    dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_cov['time'].values],
                                 'x': xr_cov.x,
                                 'y': xr_cov.y})

    print('   interpolating and saving to disk')
    t1 = time.time()

    dat_out_nans = np.zeros((dat_out.dims['time'], 
                             dat_out.dims['y'], 
                             dat_out.dims['x'])) * np.nan

    dat_out = dat_out.assign(TMP=(['time', 'y', 'x'],
                                   dat_out_nans))
    dat_out = dat_out.rename({'TMP': c})

    dat_out_ds = xr.concat([dat_out[c], xr_cov[c]], dim='time').to_dataset()
    dat_out_ds = dat_out_ds.sortby('time')
    dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

    dat_out_ds[c] = xr.concat([dat_out[c], xr_cov[c]], dim='time')
    dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})
    
    if full_smooth:
        dat_out_da = despike_ts_xr(dat_out_ds[c],
                                   dat_thresh=0.15, 
                                   mask_outliers=False,
                                   iters=2,
                                   dims=['time'])
        dat_out_da = smooth_xr(dat_out_da, 
                               dims=['time'], 
                               kwargs={'double': True, 'limit': 91})
    else:
        dat_out_da = dat_out_ds[c].interpolate_na('time').rolling(time=21, 
                                                                  center=True,
                                                                  min_periods=1).mean()
        
    dat_out_da = dat_out_da.sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
    
    dat_out_da.to_netcdf(
        os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'))

    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    del dat_out_da, dat_out_ds, dat_out, dat_out_nans
    gc.collect()
    client.restart()